In [5]:
import requests
from urllib import urlencode # This module provides a high-level interface for fetching data across the World Wide Web
from bs4 import BeautifulSoup
import re
import numpy as np
from time import sleep
from nltk.tokenize import sent_tokenize, word_tokenize

links=[]

def get_url():
    
    """Get the main searching page url"""
    base = 'https://www.indeed.com/jobs?'
    q = 'data scientist'

    for i in np.linspace(10, 50, 5, dtype= 'int'):
        page = i
        url = base + urlencode({'q':q}) + '&' + urlencode({'start':page})
        print url
        
        sleep(10) # added delay to ensure indeed server does not reject url requests
        
        """Scraping job title href links from the main searched page """
        try:
            r = requests.get(url)
        except requests.exceptions.ConnectionError: # solves for indeed server rejecting url requests
            r.status_code = "Connection refused"
            
        soup =  BeautifulSoup(r.text, 'html.parser') # parse into html
        results = soup.find_all('a', attrs={'data-tn-element':'jobTitle'})

        base_2 = 'https://www.indeed.com'

        for i in results:
            links.append(base_2 + i['href'])

    return links

In [ ]:
get_url()

https://www.indeed.com/jobs?q=data+scientist&start=10
https://www.indeed.com/jobs?q=data+scientist&start=20
https://www.indeed.com/jobs?q=data+scientist&start=30
https://www.indeed.com/jobs?q=data+scientist&start=40
https://www.indeed.com/jobs?q=data+scientist&start=50


In [3]:
print len(links)

80
